<center>

# **PREDICTIVE METHOD**<br>
# **NEURAL NETWORK**<br>

by: Ly Nguyen

</center>


In [1]:
# Import necessary libraries for this notebook: 

# Read from SQLite database and load to a pandas dataframe
import os
import sqlite3
import pandas as pd

# For using arrays 
import numpy as np

# For ML work (data preprocessing, hyperparameter tuning, Random Forest Classifier, training & testing sets, and stratified sampling)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.utils.class_weight import compute_class_weight



# For model evaluation, including explainability:  
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, balanced_accuracy_score, make_scorer
from sklearn.utils.class_weight import compute_class_weight
import statsmodels.api as sm
import shap

# For data visualization 
import matplotlib.pyplot as plt
import seaborn as sns

# For saving the model into a pkl file
import joblib



IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import keras_tuner as kt

In [3]:
# Load the saved df_prelim parquet file: 
relative_path = os.path.join("..", "src", "df_reduced.parquet")
df_reduced = pd.read_parquet(relative_path)

In [4]:
# Define X and y:
X = df_reduced.drop(columns=['delayType'])  # Use parentheses with the 'columns' argument
y = df_reduced['delayType']


In [5]:
# Perform stratified split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)


In [6]:
# One-hot encode the target for neural network
y_train_encoded = to_categorical(y_train - 1)  # Zero-indexed for NN
y_val_encoded = to_categorical(y_val - 1)
y_test_encoded = to_categorical(y_test - 1)

# Hyperparameter tuning

In [7]:
# Compute class weights
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weights = dict(enumerate(class_weights))
print("Class weights:", class_weights)

Class weights: {0: 3.1258878275777615, 1: 0.4759294477383749, 2: 1.7272973338746787}


In [8]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(hp.Int('units', min_value=64, max_value=256, step=64),
                    activation='relu', input_shape=(X_train.shape[1],)))
    model.add(Dropout(hp.Float('dropout', 0.1, 0.5, step=0.1)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [0.001, 0.0001])),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model



In [9]:
tuner = kt.Hyperband(build_model, 
                     objective='val_accuracy', 
                     max_epochs=30, 
                     factor=3)

# Hyperparameter search
tuner.search(X_train, y_train_encoded, 
             validation_data=(X_val, y_val_encoded), 
             class_weight=class_weights)

# Retrieve the best model
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
best_model = tuner.hypermodel.build(best_hps)


Reloading Tuner from .\untitled_project\tuner0.json


# Fitting the model

In [10]:
history = best_model.fit(X_train, y_train_encoded,
                         validation_data=(X_val, y_val_encoded),
                         class_weight=class_weights,
                         epochs=30,
                         batch_size=32,
                         verbose=1)


Epoch 1/30
399/399 [==============================] - 2s 3ms/step - loss: 0.9256 - accuracy: 0.4913 - val_loss: 0.9287 - val_accuracy: 0.5170
Epoch 2/30
399/399 [==============================] - 1s 3ms/step - loss: 0.8355 - accuracy: 0.5489 - val_loss: 0.9153 - val_accuracy: 0.5499
Epoch 3/30
399/399 [==============================] - 1s 2ms/step - loss: 0.8015 - accuracy: 0.5727 - val_loss: 0.8805 - val_accuracy: 0.5686
Epoch 4/30
399/399 [==============================] - 1s 2ms/step - loss: 0.7859 - accuracy: 0.5879 - val_loss: 0.8859 - val_accuracy: 0.5689
Epoch 5/30
399/399 [==============================] - 2s 4ms/step - loss: 0.7704 - accuracy: 0.5847 - val_loss: 0.9225 - val_accuracy: 0.5371
Epoch 6/30
399/399 [==============================] - 2s 4ms/step - loss: 0.7540 - accuracy: 0.5898 - val_loss: 0.8986 - val_accuracy: 0.5488
Epoch 7/30
399/399 [==============================] - 1s 2ms/step - loss: 0.7494 - accuracy: 0.5965 - val_loss: 0.9229 - val_accuracy: 0.5521
Epoch 

In [11]:
y_pred_proba = best_model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1) + 1  # Convert back to original labels


86/86 [==============================] - 0s 1ms/step


# Model Evaluation

In [12]:
# Evaluation scores
print(classification_report(y_test, y_pred))

balanced_acc = balanced_accuracy_score(y_test, y_pred)
print(f"Balanced Accuracy: {balanced_acc * 100:.2f}%")


              precision    recall  f1-score   support

           1       0.33      0.62      0.43       292
           2       0.84      0.54      0.65      1916
           3       0.37      0.67      0.47       528

    accuracy                           0.57      2736
   macro avg       0.51      0.61      0.52      2736
weighted avg       0.69      0.57      0.59      2736

Balanced Accuracy: 60.68%


# **Observation:**
- This 5th model performs worse than the 3rd model across the scores. 
- The 3rd model is the optimal one so far.